In [2]:
from population_stats import *

nacc = load_dataset_as_df("NACC")
adni = load_dataset_as_df("ADNI")
ds = stack_datasets(nacc=nacc, adni=adni)

In [3]:
ds.head()

,age,mmse,sex,apoe,educ,progress,Dataset
0,95,27,F,0.0,12,0,nacc
1,77,21,M,0.0,12,1,nacc
2,80,28,F,0.0,18,1,nacc
3,38,25,F,0.0,14,1,nacc
4,81,29,M,0.0,12,0,nacc


In [5]:
ds.to_parquet("csvs/demographics.parquet")

In [4]:
apoe = ds[["apoe","progress","Dataset"]]
sex = ds[["sex", "progress", "Dataset"]]

educ = ds[["educ", "progress", "Dataset"]]
age = ds[["age", "progress", "Dataset"]]
mmse = ds[["mmse", "progress", "Dataset"]]


with open("csvs/statistics.txt", "w") as fi:
    for x in (0,1):
        fi.write("="*50 + "\n")
        fi.write(f"Progress == {x}\n\n")
        apoe_progress = apoe.query("progress == @x").copy()
        apoe_progress = apoe_progress.drop(apoe_progress.loc[apoe_progress["apoe"] == ""].index)
        ct = pd.crosstab(apoe_progress["Dataset"], apoe_progress["apoe"], margins=True)
        cs = ChiSquare(ct)
        fi.write(str(cs))

        sex_progress = sex.query("progress == @x").copy()
        sex_progress = sex_progress.drop(sex_progress.loc[sex_progress["sex"] == ""].index)
        ct = pd.crosstab(sex_progress["Dataset"], sex_progress["sex"], margins=True)
        cs = ChiSquare(ct)
        fi.write(str(cs))

        for col in ("educ", "age", "mmse"):
            sub_df = ds[[col, "progress", "Dataset"]]
            sub_df_progress = sub_df.query("progress == @x").copy()
            mwu = MannWhitneyU(
                sub_df.query("Dataset == \"nacc\"")[col].astype("float").to_numpy(),
                sub_df.query("Dataset == \"adni\"")[col].astype("float").to_numpy(),
                "nacc",
                "adni"
                )
            fi.write(f"{col.upper()}\n\n")
            fi.write(str(mwu))